### Things We Should Address

1. **Tool Selection**  
   We will initially implement tools as standalone functions. Later, we will refactor them into an object-oriented structure for better modularity and maintainability.

2. **Prompt Design**  
   Once the workflow is functional, we will apply prompt engineering techniques—such as few-shot learning—to improve the LLM's understanding of the MINI project.

3. **Agent Structure**  
   - **Built-in React Agent vs. Custom Agent**  
     For now, we will use `create_react_agent` from LangChain to evaluate its benefits for our project.  
     - One limitation of `create_react_agent` is that the prompt template is somewhat fixed, which may reduce flexibility.  
     - For the `agent_scratchpad`, which is injected into the prompt, we don’t need to send the entire message history—just the last assistant response is sufficient.

4. **React Agent Loop**  
   - How the patient agent interacts with the React agent loop.

5. **Agoraphobia JSON Fix**  
   - The Agoraphobia module’s JSON file has been corrected and updated.


In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install openai
!pip install tiktoken
!pip install python-dotenv


In [1]:
import json
from pathlib import Path
from typing import Dict, List, Any, Tuple

def load_mini_module(file_path: str) -> Tuple[str, List[str], List[Dict[str, Any]], Dict[str, Dict[str, str]]]:
    """
    Load MINI diagnostic module from JSON file.

    Returns:
        - module_id: e.g., 'E'
        - instructions: list of string instructions
        - questions: list of question dicts
        - diagnostic_criteria: dict mapping condition to evaluation logic
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    module_id = data["module"]["id"]
    instructions = data.get("instructions", [])
    questions = data.get("questions", [])
    diagnostic_criteria = data.get("diagnosticCriteria", {})

    return module_id, instructions, questions, diagnostic_criteria

# Example usage:
# module_id, instructions, questions, criteria = load_mini_module("mini-modules/module_e.json")
# print(instructions)
# print(questions[0]['prompt'])


In [ ]:
from langchain.agents import tool
@tool
def ask_patient(question: str) -> str:
    """Ask the patient the current MINI question."""
    pass

@tool
def classify_answer(response: str) -> str:
    """Classify patient's natural-language response as 'yes', 'no', or 'unclear'."""
    pass
@tool
def explain(current_question: str) -> str:
    """Clarify or rephrase the current question to help the patient understand."""
    pass
@tool
def end_module() -> str:
    """Signal that the module has ended either due to logic or patient response."""
    pass

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

load_dotenv()

interview_prompt_template = """
You are an intelligent and empathetic interviewer conducting a psychiatric diagnostic session using the MINI system.

You operate in a deliberate loop with the following structure:

- **Thought**: Reflect on the current question, the patient's response (if available), and your reasoning about what to do next.
- **Action**: Choose a tool to use from the available actions, then return "PAUSE".
- **Observation**: Record the result returned by the tool (e.g., the patient's response, a classification, or a confirmation).
- Repeat this loop as needed to continue the interview process.

At the end of the loop, output an **Answer** — this should reflect one of the following:
- The decision about what to do next (e.g., ask the next question),
- A clarification or rephrasing of the current question,
- Or a decision to end the module.

### Your goals:
1. Ask the current question naturally using `ask_patient`.
2. Interpret the patient's response using `classify_answer`.
3. If the response is unclear or vague, use `explain` to rephrase or clarify.
4. If the logic requires termination, use `end_module`.
5. Always reason about what to do next based on the MINI module instructions.

Here are the MINI instructions for the current module:
{instructions}

You have access to the following tools:
{tools}

Remember: always reason step by step using **Thought**, take an **Action**, pause, then reflect on the **Observation**. Only then output your **Answer**.

Begin.

Interview Context:
{input}

{agent_scratchpad}
"""
prompt = PromptTemplate.from_template(interview_prompt_template)
llm = ChatOpenAI(temperature=0)

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

from langchain_core.agents import AgentAction, AgentFinish

# Initialize state
current_question_id = "E1"
intermediate_steps = []
agent_step = None
question_dict = {q["id"]: q for q in questions}

while not isinstance(agent_step, AgentFinish):
    # 1. Invoke the agent with input and intermediate steps
    question_prompt = question_dict[current_question_id]["prompt"]

    agent_step = agent.invoke({
        "input": question_prompt,
        "intermediate_steps": intermediate_steps,
    })

    # 2. If it's an action, run the tool manually
    if isinstance(agent_step, AgentAction):
        tool_name = agent_step.tool
        tool_input = agent_step.tool_input

        tool = next(t for t in tools if t.name == tool_name)
        observation = tool.invoke(tool_input)

        print(f"\nTool: {tool_name}")
        print(f"Input: {tool_input}")
        print(f"Observation: {observation}")

        # Append to scratchpad
        intermediate_steps.append((agent_step, observation))

# 3. Done — agent produced final answer
if isinstance(agent_step, AgentFinish):
    print("\nFinal Answer:", agent_step.return_values["output"])



In [ ]:
from openai import OpenAI
client = OpenAI()
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
